<h1 style="color:darkmagenta; text-align:center; font-family:Cursive">
    <b>Salary Prediction
    <b><br>of
    <b><br>Data Science Job Salary
</h1>

## <div><p style="color:#b50264; font-family:Cursive"><b>🎯 Notebook Goal</p></div>

1. **Predicting the salary** of the Data Science Job market. 💸

## <div><p style="color:#b50264; font-family:Cursive"><b>🏷️ Table of Contents</p></div>
<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">

   1. [Import Necessary Libraries](#1)
   2. [Getting Data](#2)

<a id="1"></a>
## <div><p style="color:darkmagenta; font-family:Cursive"><b>1. Import Necessary Libraries</p></div>

In [1]:
# data
import pandas as pd
import numpy as np

<a id="2"></a>
## <div><p style="color:darkmagenta; font-family:Cursive"><b>2. Getting Data</p></div>

In [2]:
# read data and create DataFrame
df = pd.read_csv("../data_given/ds_salaries.csv", index_col=0)
df.head(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
